In [2]:
import pandas as pd
import numpy as np
from functools import reduce
pd.set_option('future.no_silent_downcasting', True)

In [3]:
# FAO Data
fao_fbs = pd.read_csv(r"..\data\FAOSTAT\processed\fao_fbs_relevant.csv")
fao_fs = pd.read_csv(r"..\data\FAOSTAT\processed\fao_fs_relevant.csv")
fao_emissions = pd.read_csv(r"..\data\FAOSTAT\processed\fao_emissions_relevant.csv")
fao_cpi = pd.read_csv(r"..\data\FAOSTAT\processed\fao_cpi_relevant.csv")
fao_population = pd.read_csv(r"..\data\FAOSTAT\processed\fao_population_relevant.csv")
fao_prices = pd.read_csv(r"..\data\FAOSTAT\processed\fao_prices_relevant.csv")
fao_production_indices = pd.read_csv(r"..\data\FAOSTAT\processed\fao_production_indices_relevant.csv")
fao_fbs_item_group = pd.read_csv(r"..\data\FAOSTAT\metadata\fbs_item_group.csv")

In [4]:
# Other Data
f_waste = pd.read_csv(r"../data/processed/unep_waste_merged.csv")
fsi_data = pd.read_csv(r"../data/processed/fsi_data_melted.csv")
# asean_sdg = pd.read_excel(r"../data/SDG From ASEANstats.xlsx")
# un_sdg = pd.read_excel(r"../data/SDG From UNSDG (2, 11, 12, 13).xlsx")
gfsi_data = pd.read_csv(r"../data/processed/gfsi_data_melted.csv")
fsi_score = pd.read_csv(r"../data/processed/fsi_score_melted.csv")
gfsi_score = pd.read_csv(r"../data/processed/gfsi_score_melted.csv")

# Final Data Preparation

## Changing Laos Official Name in UNEP Dataset for Consistency

In [5]:
fao_fs['Year']

0         2000-2002
1         2001-2003
2         2002-2004
3         2003-2005
4         2004-2006
            ...    
283875    2016-2018
283876    2017-2019
283877    2018-2020
283878    2019-2021
283879    2020-2022
Name: Year, Length: 283880, dtype: object

In [6]:
# Replace all Lao People's Dem. Rep. with Lao People's Democratic Republic
f_waste['Country'] = f_waste['Country'].replace('Lao People\'s Dem. Rep.', 'Lao People\'s Democratic Republic')

## Remove Food Item Group Column from FAO Dataset to Ensure Consistency for Detailed Analysis

In [7]:
fao_fbs_item_group

,Item Group Code,Item Group,Item Code,Item,Factor,CPC Code,HS Code,HS07 Code,HS12 Code
0,2901,Cereals - Excluding Beer,2511,Wheat and products,1,NaN,NaN,NaN,NaN
1,2903,Cereals - Excluding Beer,2511,Wheat and products,1,NaN,NaN,NaN,NaN
2,2905,Grand Total,2511,Wheat and products,1,NaN,NaN,NaN,NaN
3,2901,Cereals - Excluding Beer,2807,Rice and products,1,NaN,NaN,NaN,NaN
4,2903,Cereals - Excluding Beer,2807,Rice and products,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
289,2901,Miscellaneous,2680,Infant food,1,NaN,NaN,NaN,NaN
290,2903,Miscellaneous,2680,Infant food,1,NaN,NaN,NaN,NaN
291,2903,Grand Total,2899,Miscellaneous,1,NaN,NaN,NaN,NaN
292,2928,Grand Total,2899,Miscellaneous,1,NaN,NaN,NaN,NaN


In [8]:
fao_fbs

,Area,Item,Element,Year,Unit,Value,Flag
0,Afghanistan,Population,Total Population - Both sexes,2010,1000 No,28189.67,X
1,Afghanistan,Population,Total Population - Both sexes,2011,1000 No,29249.16,X
2,Albania,Population,Total Population - Both sexes,2010,1000 No,2913.40,X
3,Albania,Population,Total Population - Both sexes,2011,1000 No,2900.65,X
4,Afghanistan,Population,Total Population - Both sexes,2012,1000 No,30466.48,X
...,...,...,...,...,...,...,...
4660695,World,Miscellaneous,Fat supply quantity (t),2018,t,68588.87,E
4660696,World,Miscellaneous,Fat supply quantity (t),2019,t,74915.46,E
4660697,World,Miscellaneous,Fat supply quantity (t),2020,t,75849.36,E
4660698,World,Miscellaneous,Fat supply quantity (t),2021,t,74733.79,E


In [9]:
# Remove rows from fao_fbs where 'Item' matches any value in fao_fbs_item_group['Item Group']
items_to_remove = set(fao_fbs_item_group['Item Group'].unique())

# Detect rows where 'Item' matches any value in fao_fbs_item_group['Item Group']
matching_rows = fao_fbs['Item'].isin(items_to_remove)
print(f"Number of rows to remove: {matching_rows.sum()}")
fao_fbs[matching_rows][['Area', 'Item', 'Year']]

Number of rows to remove: 976493


,Area,Item,Year
100,Afghanistan,Grand Total,2010
112,Albania,Grand Total,2010
114,Afghanistan,Grand Total,2011
127,Algeria,Grand Total,2010
130,Afghanistan,Grand Total,2012
...,...,...,...
4660695,World,Miscellaneous,2018
4660696,World,Miscellaneous,2019
4660697,World,Miscellaneous,2020
4660698,World,Miscellaneous,2021


In [10]:
# Remove rows
fao_fbs = fao_fbs[~matching_rows]

# Exploratory Data Analysis (EDA)

## Food Insecurity vs. Food Waste

### Food Insecurity in Urban Area vs. Household Food Waste (Archive)

In [11]:
# Define the list of ASEAN countries for consistent filtering across all datasets.
asean_countries = [
    'Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao People\'s Democratic Republic',
    'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Viet Nam'
]

# Process Urban Population Data
max_actual_year = fao_population[fao_population['Year'] <= 2023]['Year'].max()
population_urban = fao_population[
    (fao_population['Area'].isin(asean_countries)) &
    (fao_population['Element'] == 'Urban population') &
    (fao_population['Year'] == 2024)
].copy()
population_urban['Value'] *= 1000  # Convert from '1000 persons' to persons
population_urban = population_urban[['Area', 'Year', 'Value']].rename(columns={
    'Area': 'Country',
    'Year': 'Population Year',
    'Value': 'Urban Population'
})

# Process Urban Food Insecurity Data
fs_urban_insecurity = fao_fs[
    (fao_fs['Area'].isin(asean_countries)) &
    (fao_fs['Item'] == 'Prevalence of moderate or severe food insecurity in the total population (percent) (3-year average)') &
    (fao_fs['Year'].astype(str) == '2021-2023') &
    (fao_fs['Element'] == 'Value')
].copy()
fs_urban_insecurity = fs_urban_insecurity[['Area', 'Year', 'Value']].rename(columns={
    'Area': 'Country',
    'Year': 'FS Year',
    'Value': 'Urban Food Insecurity (%)'
})

# Process Food Waste Data
waste_household = f_waste[f_waste['Country'].isin(asean_countries)][['Country', 'Household estimate (kg/capita/year)']].rename(
    columns={'Household estimate (kg/capita/year)': 'Household Waste (kg/capita/yr)'}
)

# Merge Datasets to Create the Urban Paradox DataFrame
urban_paradox = population_urban.merge(fs_urban_insecurity, on='Country', how='left').merge(waste_household, on='Country', how='left')
urban_paradox['Total Household Waste (tonnes/yr)'] = (urban_paradox['Urban Population'] * urban_paradox['Household Waste (kg/capita/yr)']) / 1000
urban_paradox = urban_paradox.loc[:, ~urban_paradox.columns.str.contains('Year')]

urban_paradox_fmt = urban_paradox.copy()
urban_paradox_fmt['Urban Population'] = urban_paradox_fmt['Urban Population'].apply(lambda x: f"{x:,.0f}")
urban_paradox_fmt['Total Household Waste (tonnes/yr)'] = urban_paradox_fmt['Total Household Waste (tonnes/yr)'].apply(lambda x: f"{x:,.0f}")

print("--- Urban Paradox Data Ready for Visualization ---")
urban_paradox_fmt

--- Urban Paradox Data Ready for Visualization ---


,Country,Urban Population,Urban Food Insecurity (%),Household Waste (kg/capita/yr),Total Household Waste (tonnes/yr)
0,Brunei Darussalam,"368,738",NaN,76,"28,024"
1,Cambodia,"4,581,944",50.5,85,"389,465"
2,Indonesia,"167,176,839",4.9,53,"8,860,372"
3,Lao People's Democratic Republic,"2,933,677",36.3,89,"261,097"
4,Malaysia,"27,360,379",16.7,81,"2,216,191"
5,Myanmar,"18,372,626",32,78,"1,433,065"
6,Philippines,"56,434,179",44.1,26,"1,467,289"
7,Singapore,"6,119,203",7.7,68,"416,106"
8,Thailand,"37,838,857",7.2,86,"3,254,142"
9,Viet Nam,"40,975,863",10.8,72,"2,950,262"


### Food Insecurity Population vs. Total Food Waste

In [12]:
# Define the list of ASEAN countries for consistent filtering across all datasets.
asean_countries = [
    'Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao People\'s Democratic Republic',
    'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Viet Nam'
]

# Process Population Data
population_asean = fao_population[
    (fao_population['Area'].isin(asean_countries)) &
    (fao_population['Element'] == 'Total Population - Both sexes') &
    (fao_population['Year'] == 2022)
].copy()
population_asean['Value'] *= 1000  # Convert from '1000 persons' to persons
population_asean = population_asean[['Area', 'Year', 'Value']].rename(columns={
    'Area': 'Country',
    'Year': 'Population Year',
    'Value': 'Population'
})

# Extract ASEAN Food Insecurity Data (prevalence % and number in millions, 2021-2023)
fs_asean_insecurity = fao_fs[
    (fao_fs['Area'].isin(asean_countries)) &
    (fao_fs['Item'].isin([
        'Prevalence of moderate or severe food insecurity in the total population (percent) (3-year average)',
        'Number of moderately or severely food insecure people (million) (3-year average)'
    ])) &
    (fao_fs['Year'].astype(str) == '2021-2023') &
    (fao_fs['Element'] == 'Value')
][['Area', 'Item', 'Year', 'Value']].rename(columns={'Area': 'Country', 'Year': 'FS Year'})

# Pivot to columns: prevalence (%) and number (people)
fs_asean_pivot = fs_asean_insecurity.pivot(index=['Country', 'FS Year'], columns='Item', values='Value').reset_index()
fs_asean_pivot.columns.name = None
fs_asean_pivot = fs_asean_pivot.rename(columns={
    'Prevalence of moderate or severe food insecurity in the total population (percent) (3-year average)': 'Food Insecurity (%)',
    'Number of moderately or severely food insecure people (million) (3-year average)': 'Food Insecure Population'
})
fs_asean_pivot['Food Insecurity (%)'] = fs_asean_pivot['Food Insecurity (%)'].astype(float)
fs_asean_pivot['Food Insecure Population'] = fs_asean_pivot['Food Insecure Population'].astype(float) * 1_000_000

# Load total food waste data (household + retail + food service)
waste_cols = [
    'Household estimate (tonnes/year)',
    'Retail estimate (tonnes/year)',
    'Food service estimate (tonnes/year)'
]
total_waste = f_waste[f_waste['Country'].isin(asean_countries)][['Country'] + waste_cols].copy()
total_waste = total_waste.replace('-', pd.NA).infer_objects(copy=False)
for col in waste_cols:
    total_waste[col] = total_waste[col].astype('Int64')

# Impute missing Food service estimate (tonnes/year) using average ratio to (household + retail)
cols = [
    'Household estimate (tonnes/year)',
    'Retail estimate (tonnes/year)',
    'Food service estimate (tonnes/year)'
]
total_waste[cols] = total_waste[cols].apply(pd.to_numeric, errors='coerce')
mask = total_waste[cols].notna().all(axis=1)
avg_ratio = (
    total_waste.loc[mask, 'Food service estimate (tonnes/year)'] /
    (total_waste.loc[mask, 'Household estimate (tonnes/year)'] + total_waste.loc[mask, 'Retail estimate (tonnes/year)'])
).mean()
missing = total_waste['Food service estimate (tonnes/year)'].isna()
sum_hr = total_waste.loc[missing, 'Household estimate (tonnes/year)'] + total_waste.loc[missing, 'Retail estimate (tonnes/year)']
total_waste.loc[missing, 'Food service estimate (tonnes/year)'] = (sum_hr * avg_ratio).round().astype('Int64')

total_waste['Total Food Waste (tonnes/year)'] = total_waste[waste_cols].sum(axis=1)

# Merge Datasets to Create the Insecurity vs. Food Waste DataFrame
food_insec_asean = population_asean.merge(fs_asean_pivot, on='Country', how='left').merge(total_waste, on='Country', how='left')
food_insec_asean = food_insec_asean.loc[:, ~food_insec_asean.columns.str.contains('Year')]
food_insec_asean = food_insec_asean[['Country', 'Population', 'Food Insecure Population', 'Food Insecurity (%)', 'Total Food Waste (tonnes/year)']]

food_insec_asean_fmt = food_insec_asean.copy()
food_insec_asean_fmt['Population'] = food_insec_asean_fmt['Population'].apply(lambda x: f"{x:,.0f}")
food_insec_asean_fmt['Total Food Waste (tonnes/year)'] = food_insec_asean_fmt['Total Food Waste (tonnes/year)'].apply(lambda x: f"{x:,.0f}")
food_insec_asean_fmt['Food Insecure Population'] = food_insec_asean_fmt['Food Insecure Population'].apply(lambda x: f"{x:,.0f}")

print("--- Insecurity vs. Food Waste Data Ready for Visualization ---")
food_insec_asean_fmt

--- Insecurity vs. Food Waste Data Ready for Visualization ---


,Country,Population,Food Insecure Population,Food Insecurity (%),Total Food Waste (tonnes/year)
0,Brunei Darussalam,"455,370",nan,NaN,"70,577"
1,Cambodia,"17,201,724","8,500,000",50.5,"2,838,261"
2,Indonesia,"278,830,529","13,600,000",4.9,"39,535,795"
3,Lao People's Democratic Republic,"7,559,007","2,700,000",36.3,"1,310,729"
4,Malaysia,"34,695,493","5,700,000",16.7,"7,110,919"
5,Myanmar,"53,756,787","17,400,000",32.0,"8,804,564"
6,Philippines,"113,964,339","51,000,000",44.1,"12,728,801"
7,Singapore,"5,649,885","500,000",7.7,"893,813"
8,Thailand,"71,735,329","5,200,000",7.2,"12,636,704"
9,Viet Nam,"99,680,655","10,600,000",10.8,"15,384,854"


## Estimated Financial Loss (Hybrid Price Filling) From Food Loss

In [13]:
# Filter the Food Balance Sheets for the 'Losses' element across all commodities
# in ASEAN countries. We will keep the 'Year' to be explicit about the timeframe.
loss_volume = fao_fbs[
    (fao_fbs['Area'].isin(asean_countries)) &
    (fao_fbs['Element'] == 'Losses') &
    (fao_fbs['Year'] == 2022) &
    (~fao_fbs['Item'].str.strip().str.lower().str.contains('total'))
].copy()
loss_volume['Value'] = loss_volume['Value'] * 1000
loss_volume = loss_volume.sort_values('Year').drop_duplicates(['Area', 'Item'], keep='last')
loss_volume = loss_volume[['Area', 'Item', 'Year', 'Value']].rename(columns={
    'Year': 'Loss Year',
    'Value': 'Losses (tonnes)'
})

# Filter the Producer Prices dataset for the USD/tonne value across all commodities
# in ASEAN, also keeping the 'Year'.
producer_prices = fao_prices[
    (fao_prices['Area'].isin(asean_countries)) &
    (fao_prices['Element'] == 'Producer Price (USD/tonne)')
].copy()
# For each (Area, Item), find the latest available price (any year)
producer_prices = (
    producer_prices
    .sort_values(['Area', 'Item', 'Year'], ascending=[True, True, False])
    .drop_duplicates(['Area', 'Item'], keep='first')
    .loc[:, ['Area', 'Item', 'Year', 'Value']]
    .rename(columns={'Year': 'Price Year', 'Value': 'Price (USD/tonne)'})
)

# Manual mapping from loss_volume items to producer_prices items (standardized, lowercased)
item_mapping = {
    'eggs': 'hen eggs in shell, fresh',
    'milk - excluding butter': 'raw milk of cattle',
    'fats, animals, raw': 'palm oil',
    'animal fats': 'palm oil',
    'rice and products': 'rice',
    'wheat and products': 'wheat',
    'cereals - excluding beer': 'cereals n.e.c.',
    'cassava and products': 'cassava, fresh',
    'starchy roots': 'edible roots and tubers with high starch or inulin content, n.e.c., fresh',
    'cereals, other': 'cereals n.e.c.',
    'sorghum and products': 'sorghum',
    'roots, other': 'edible roots and tubers with high starch or inulin content, n.e.c., fresh',
    'sweet potatoes': 'sweet potatoes',
    'potatoes and products': 'potatoes',
    'sugar crops': 'other sugar crops n.e.c.',
    'sugar cane': 'sugar cane',
    'yams': 'yams',
    'pulses': 'other pulses n.e.c.',
    'sweeteners, other': 'natural honey',
    'honey': 'natural honey',
    'sugar & sweeteners': 'sugar cane',
    'peas': 'peas, dry',
    'beans': 'beans, dry',
    'pulses, other and products': 'other pulses n.e.c.',
    'oilcrops': 'oil palm fruit',
    'treenuts': 'other nuts (excluding wild edible nuts and groundnuts), in shell, n.e.c.',
    'nuts and products': 'other nuts (excluding wild edible nuts and groundnuts), in shell, n.e.c.',
    'groundnuts': 'groundnuts, excluding shelled',
    'soyabeans': 'soya beans',
    'sunflower seed': 'sunflower seed',
    'coconuts - incl copra': 'coconuts, in shell',
    'cottonseed': 'cotton seed',
    'rape and mustardseed': 'rape or colza seed',
    'sesame seed': 'sesame seed',
    'oilcrops, other': 'other oil seeds, n.e.c.',
    'palm kernels': 'palm kernels',
    'vegetables': 'other vegetables, fresh n.e.c.',
    'palmkernel oil': 'palm kernels',
    'vegetable oils': 'oil palm fruit',
    'bananas': 'bananas',
    'onions': 'onions and shallots, dry (excluding dehydrated)',
    'fruits - excluding wine': 'other fruits, n.e.c.',
    'tomatoes and products': 'tomatoes',
    'grapefruit and products': 'pomelos and grapefruits',
    'vegetables, other': 'other vegetables, fresh n.e.c.',
    'lemons, limes and products': 'lemons and limes',
    'oranges, mandarines': 'oranges',
    'palm oil': 'palm oil',
    'pineapples and products': 'pineapples',
    'stimulants': 'other stimulant, spice and aromatic crops, n.e.c.',
    'plantains': 'plantains',
    'apples and products': 'apples',
    'fruits, other': 'other fruits, n.e.c.',
    'citrus, other': 'other citrus fruit, n.e.c.',
    'grapes and products (excl wine)': 'grapes',
    'coffee and products': 'coffee, green',
    'dates': 'dates',
    'spices': 'other stimulant, spice and aromatic crops, n.e.c.',
    'tea (including mate)': 'tea leaves',
    'spices, other': 'other stimulant, spice and aromatic crops, n.e.c.',
    'pimento': 'chillies and peppers, green (capsicum spp. and pimenta spp.)',
    'cocoa beans and products': 'cocoa beans',
    'pepper': 'pepper (piper spp.), raw',
    'cloves': 'cloves (whole stems), raw',
    'offals, edible': 'other meat of mammals, fresh or chilled',
    'offals': 'other meat of mammals, fresh or chilled',
    'meat': 'meat of chickens, fresh or chilled',
    'poultry meat': 'meat of chickens, fresh or chilled',
    'pigmeat': 'meat of pig with the bone, fresh or chilled',
    'millet and products': 'millet',
    'maize and products': 'maize (corn)',
    'barley and products': 'barley'
}

# Standardize and map items in loss_volume
loss_volume['Item_std'] = loss_volume['Item'].str.strip().str.lower()
loss_volume['Item_mapped'] = loss_volume['Item_std'].map(item_mapping)

# Only keep rows with a mapped item
loss_volume_mapped = loss_volume[loss_volume['Item_mapped'].notnull()].copy()

# Standardize producer_prices item names
producer_prices['Item_std'] = producer_prices['Item'].str.strip().str.lower()

# Hybrid approach: Fill missing prices with ASEAN and global averages for each item
# Calculate ASEAN average price for each item (excluding the country in question)
asean_avg_prices = (
    producer_prices.groupby('Item_std')['Price (USD/tonne)']
    .mean()
    .rename('ASEAN_Avg_Price')
    .reset_index()
)

# Calculate global average price for each item (from all available data)
global_prices = fao_prices[
    (fao_prices['Element'] == 'Producer Price (USD/tonne)')
].copy()
global_prices['Item_std'] = global_prices['Item'].str.strip().str.lower()
global_avg_prices = (
    global_prices.groupby('Item_std')['Value']
    .mean()
    .rename('Global_Avg_Price')
    .reset_index()
)

# Merge ASEAN and global averages into loss_volume_mapped
loss_volume_mapped = loss_volume_mapped.merge(
    asean_avg_prices, left_on='Item_mapped', right_on='Item_std', how='left', suffixes=('', '_ASEAN')
)
loss_volume_mapped = loss_volume_mapped.merge(
    global_avg_prices, left_on='Item_mapped', right_on='Item_std', how='left', suffixes=('', '_GLOBAL')
)

# Merge with producer_prices to get country-specific price if available
merged = pd.merge(
    loss_volume_mapped, producer_prices,
    left_on=['Area', 'Item_mapped'],
    right_on=['Area', 'Item_std'],
    how='left',
    suffixes=('', '_country')
)

# Choose price: country > ASEAN > global
def choose_price(row):
    if pd.notnull(row['Price (USD/tonne)']):
        return row['Price (USD/tonne)']
    elif pd.notnull(row['ASEAN_Avg_Price']):
        return row['ASEAN_Avg_Price']
    else:
        return row['Global_Avg_Price']

merged['Final_Price'] = merged.apply(choose_price, axis=1)
merged = merged[merged['Final_Price'].notnull()].copy()

merged['Financial Loss (USD)'] = merged['Losses (tonnes)'] * merged['Final_Price']

financial_loss_df_detailed = merged.rename(columns={'Area': 'Country'})
financial_loss_df_detailed = financial_loss_df_detailed.sort_values('Financial Loss (USD)', ascending=False)

print("--- Comprehensive Financial Loss Data (Hybrid Price Filling) ---")
financial_loss_df_detailed[['Country', 'Item', 'Loss Year', 'Final_Price', 'Losses (tonnes)', 'Financial Loss (USD)']]

--- Comprehensive Financial Loss Data (Hybrid Price Filling) ---


,Country,Item,Loss Year,Final_Price,Losses (tonnes),Financial Loss (USD)
155,Indonesia,Palm kernels,2022,281.400000,35611000.0,1.002094e+10
17,Viet Nam,Rice and products,2022,326.700000,3808000.0,1.244074e+09
67,Indonesia,Sugar cane,2022,699.983333,1732000.0,1.212371e+09
146,Viet Nam,"Vegetables, other",2022,755.525000,1573000.0,1.188441e+09
22,Indonesia,Rice and products,2022,389.000000,3006000.0,1.169334e+09
...,...,...,...,...,...,...
249,Cambodia,Pepper,2022,2416.100000,0.0,0.000000e+00
239,Thailand,Pepper,2022,6525.500000,0.0,0.000000e+00
244,Philippines,Pepper,2022,5002.500000,0.0,0.000000e+00
276,Viet Nam,Millet and products,2022,2522.700000,0.0,0.000000e+00


In [14]:
# Summarize Financial Losses with formatted output
financial_loss_by_country = financial_loss_df_detailed.groupby('Country')['Financial Loss (USD)'].sum().reset_index()
financial_loss_by_country = financial_loss_by_country.sort_values('Financial Loss (USD)', ascending=False)

# Format the numbers with commas and no scientific notation
financial_loss_by_country_fmt = financial_loss_by_country.copy()
financial_loss_by_country_fmt['Financial Loss (USD)'] = financial_loss_by_country_fmt['Financial Loss (USD)'].apply(lambda x: f"{x:,.0f}")

print("--- Total Financial Loss by Country (USD) ---")
financial_loss_by_country_fmt

--- Total Financial Loss by Country (USD) ---


,Country,Financial Loss (USD)
1,Indonesia,"15,672,312,286"
7,Viet Nam,"4,562,415,189"
4,Myanmar,"3,619,593,833"
6,Thailand,"2,358,256,014"
5,Philippines,"1,905,997,559"
3,Malaysia,"946,021,280"
2,Lao People's Democratic Republic,"613,789,080"
0,Cambodia,"583,373,547"


### Archived
> **Note:**  
The code below was previously used to debug unwanted output during financial loss calculation. It is no longer needed, as the desired output has been achieved.

In [15]:
print(loss_volume.shape)
print(financial_loss_df_detailed.shape)

(286, 6)
(286, 16)


In [16]:
# Diagnostic: Check for loss_volume rows that are dropped after mapping and merging
print(f"Rows in loss_volume: {len(loss_volume)}")
print(f"Rows in loss_volume_mapped: {len(loss_volume_mapped)}")
print(f"Rows in financial_loss_df_detailed: {len(financial_loss_df_detailed)}")

Rows in loss_volume: 286
Rows in loss_volume_mapped: 286
Rows in financial_loss_df_detailed: 286


In [17]:
# Diagnostic: Show which standardized item names from loss_volume are not present in the mapping dictionary
unmapped_items = set(loss_volume['Item_std'].unique()) - set(item_mapping.keys())
print("Items in loss_volume that are NOT present in item_mapping:")
print(sorted(unmapped_items))
print(f"Count: {len(unmapped_items)}")

# Diagnostic: Show which mapped values are not present in the price data (global_avg_prices['Item_std'])
mapped_values = set(item_mapping.values())
price_items = set(global_avg_prices['Item_std'].unique())
not_in_price = mapped_values - price_items
print("\nMapped values NOT present in price data (global_avg_prices['Item_std']):")
print(sorted(not_in_price))
print(f"Count: {len(not_in_price)}")

Items in loss_volume that are NOT present in item_mapping:
[]
Count: 0

Mapped values NOT present in price data (global_avg_prices['Item_std']):
[]
Count: 0


In [18]:
# For each item in not_in_price, check if it exists in fao_prices['Item'] with any other element (not 'Producer Price (USD/tonne)')
not_in_price_items = list(not_in_price)
other_element_rows = []

for item in not_in_price_items:
    # Find all rows in fao_prices where Item_std matches and Element is NOT 'Producer Price (USD/tonne)'
    mask = (
        (fao_prices['Item'].str.strip().str.lower() == item)
        & (fao_prices['Element'] != 'Producer Price (USD/tonne)')
    )
    matches = fao_prices[mask]
    if not matches.empty:
        other_element_rows.append((item, matches['Element'].unique().tolist()))

# Print the results
for item, elements in other_element_rows:
    print(f"Item '{item}' found in fao_prices with elements (not Producer Price): {elements}")

if not other_element_rows:
    print("No not_in_price items found in fao_prices with other elements.")

No not_in_price items found in fao_prices with other elements.


## Estimated Social Impact of Redistributing Edible Food Waste

### Household Waste Only

In [19]:
# Assumptions for estimation

# ASEAN average from Indonesia, Malaysia, Thailand studies (edible fraction, kcal/kg)
EDIBLE_WASTE_PERCENTAGE = 0.38  # Edible portion of household waste
KCAL_PER_KG_WASTE = 1477        # Caloric value per kg edible waste

asean_countries = [
    'Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao People\'s Democratic Republic',
    'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Viet Nam'
]

# Load household waste data
household_waste = f_waste[f_waste['Country'].isin(asean_countries)][['Country', 'Household estimate (tonnes/year)']].copy()
household_waste['Household estimate (tonnes/year)'] = household_waste['Household estimate (tonnes/year)'].astype(float)

# Latest daily energy requirement per country
daily_req = fao_fs[
    (fao_fs['Area'].isin(asean_countries)) &
    (fao_fs['Item'] == 'Minimum dietary energy requirement  (kcal/cap/day)') &
    (fao_fs['Element'] == 'Value') &
    (fao_fs['Year'].astype(str) == '2021-2023') 
].copy()
daily_req['Value'] = daily_req['Value'].astype(float)
daily_req = daily_req[['Area', 'Value']].rename(columns={'Value': 'Daily Caloric Requirement (kcal)'})

# Latest food insecure population per country
insecure_pop = fao_fs[
    (fao_fs['Area'].isin(asean_countries)) &
    (fao_fs['Item'] == 'Number of moderately or severely food insecure people (million) (3-year average)') &
    (fao_fs['Element'] == 'Value') &
    (fao_fs['Year'].astype(str) == '2021-2023') 
].copy()
insecure_pop['Value'] = insecure_pop['Value'].astype(float) * 1_000_000
insecure_pop = insecure_pop[['Area', 'Value']].rename(columns={'Value': 'Food Insecure Population'})

# Calculate potential people fed from edible food waste
social_impact_df = pd.merge(household_waste, daily_req, left_on='Country', right_on='Area')
social_impact_df['Edible Waste (kg/yr)'] = social_impact_df['Household estimate (tonnes/year)'] * 1000 * EDIBLE_WASTE_PERCENTAGE
social_impact_df['Total Wasted Calories (kcal/yr)'] = social_impact_df['Edible Waste (kg/yr)'] * KCAL_PER_KG_WASTE
social_impact_df['Annual Caloric Requirement (kcal/yr)'] = social_impact_df['Daily Caloric Requirement (kcal)'] * 365
social_impact_df['People Fed Annually (from waste)'] = (social_impact_df['Total Wasted Calories (kcal/yr)'] / social_impact_df['Annual Caloric Requirement (kcal/yr)']).round(0)

# Merge with food insecure population for context
social_impact_df = pd.merge(social_impact_df, insecure_pop, left_on='Country', right_on='Area', how='left')
social_impact_df['Insecure Population Coverage (%)'] = (social_impact_df['People Fed Annually (from waste)'] / social_impact_df['Food Insecure Population']) * 100

# Format output
social_impact_household = social_impact_df[[
    'Country',
    'People Fed Annually (from waste)',
    'Food Insecure Population',
    'Insecure Population Coverage (%)'
]].sort_values('People Fed Annually (from waste)', ascending=False).reset_index(drop=True)

social_impact_household_fmt = social_impact_household.copy()
social_impact_household_fmt['People Fed Annually (from waste)'] = social_impact_household['People Fed Annually (from waste)'].map('{:,.0f}'.format)
social_impact_household_fmt['Food Insecure Population'] = social_impact_household['Food Insecure Population'].map('{:,.0f}'.format)
social_impact_household_fmt['Insecure Population Coverage (%)'] = social_impact_household['Insecure Population Coverage (%)'].map('{:.2f}%'.format)

print("--- Estimated Social Impact of Redistributing Edible Food Waste ---")
social_impact_household_fmt

--- Estimated Social Impact of Redistributing Edible Food Waste ---


,Country,People Fed Annually (from waste),Food Insecure Population,Insecure Population Coverage (%)


### Total Food Waste

In [132]:
# Assumptions for estimation

# ASEAN-wide assumptions for edible waste composition and caloric density
EDIBLE_WASTE_PERCENTAGE = 0.40  # Edible fraction of consumer-level waste
KCAL_PER_KG_WASTE = 1566        # Caloric value per kg edible waste

asean_countries = [
    'Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao People\'s Democratic Republic',
    'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Viet Nam'
]

# Latest daily energy requirement per country
daily_req = fao_fs[
    (fao_fs['Area'].isin(asean_countries)) &
    (fao_fs['Item'] == 'Minimum dietary energy requirement  (kcal/cap/day)') &
    (fao_fs['Element'] == 'Value') &
    (fao_fs['Year'] == '2022')
].copy()
daily_req['Value'] = daily_req['Value'].astype(float)
daily_req = daily_req[['Area', 'Value']].rename(columns={'Value': 'Daily Caloric Requirement (kcal)'})

# Latest food insecure population per country
insecure_pop = fao_fs[
    (fao_fs['Area'].isin(asean_countries)) &
    (fao_fs['Item'] == 'Number of moderately or severely food insecure people (million) (3-year average)') &
    (fao_fs['Element'] == 'Value') &
    (fao_fs['Year'].astype(str) == '2021-2023') 
].copy()
insecure_pop['Value'] = insecure_pop['Value'].astype(float) * 1_000_000
insecure_pop = insecure_pop[['Area', 'Value']].rename(columns={'Value': 'Food Insecure Population'})

# Calculate potential people fed from edible food waste (using total waste)
social_impact_df = pd.merge(total_waste, daily_req, left_on='Country', right_on='Area')
social_impact_df['Edible Waste (kg/yr)'] = social_impact_df['Total Food Waste (tonnes/year)'] * 1000 * EDIBLE_WASTE_PERCENTAGE
social_impact_df['Total Wasted Calories (kcal/yr)'] = social_impact_df['Edible Waste (kg/yr)'] * KCAL_PER_KG_WASTE
social_impact_df['Annual Caloric Requirement (kcal/yr)'] = social_impact_df['Daily Caloric Requirement (kcal)'] * 365
social_impact_df['People Fed Annually (from waste)'] = (social_impact_df['Total Wasted Calories (kcal/yr)'] / social_impact_df['Annual Caloric Requirement (kcal/yr)']).round(0)

# Merge with food insecure population for context
social_impact_df = pd.merge(social_impact_df, insecure_pop, left_on='Country', right_on='Area', how='left')
social_impact_df['Insecure Population Coverage (%)'] = (social_impact_df['People Fed Annually (from waste)'] / social_impact_df['Food Insecure Population']) * 100

# Format output
social_impact_total = social_impact_df[[
    'Country',
    'People Fed Annually (from waste)',
    'Food Insecure Population',
    'Insecure Population Coverage (%)'
]].sort_values('People Fed Annually (from waste)', ascending=False).reset_index(drop=True)

social_impact_total_fmt = social_impact_total.copy()
social_impact_total_fmt['People Fed Annually (from waste)'] = social_impact_total['People Fed Annually (from waste)'].map('{:,.0f}'.format)
social_impact_total_fmt['Food Insecure Population'] = social_impact_total['Food Insecure Population'].map('{:,.0f}'.format)
social_impact_total_fmt['Insecure Population Coverage (%)'] = social_impact_total['Insecure Population Coverage (%)'].map('{:.2f}%'.format)

print("--- Estimated Social Impact of Redistributing Edible Food Waste ---")
social_impact_total_fmt

--- Estimated Social Impact of Redistributing Edible Food Waste ---


,Country,People Fed Annually (from waste),Food Insecure Population,Insecure Population Coverage (%)
0,Indonesia,"37,820,470","13,600,000",278.09%
1,Viet Nam,"14,766,744","10,600,000",139.31%
2,Philippines,"12,482,694","51,000,000",24.48%
3,Thailand,"11,480,498","5,200,000",220.78%
4,Myanmar,"8,385,172","17,400,000",48.19%
5,Malaysia,"6,596,490","5,700,000",115.73%
6,Cambodia,"2,730,338","8,500,000",32.12%
7,Lao People's Democratic Republic,"1,232,562","2,700,000",45.65%
8,Singapore,"802,265","500,000",160.45%
9,Brunei Darussalam,"64,702",nan,nan%


## Estimated Environmental Impact of Food Loss and Waste

### Method: Using GHG Emissions Factors of food loss/waste

In [133]:
# Emission factor: 2.5 tonnes CO2eq per tonne of food loss/waste
GHG_EMISSION_FACTOR = 2.5

asean_countries = [
    'Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao People\'s Democratic Republic',
    'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Viet Nam'
]

total_loss_per_country = loss_volume.groupby('Area')['Losses (tonnes)'].sum().reset_index()
total_loss_per_country = total_loss_per_country.rename(columns={'Area': 'Country', 'Losses (tonnes)': 'Total Food Loss (tonnes/year)'})
# Filling Singapore's total food loss based on Singapore Environment Council Report
total_loss_per_country = pd.concat([total_loss_per_country,
                                    pd.DataFrame([{'Country': 'Singapore', 'Total Food Loss (tonnes/year)': 342000}])
                                    ], ignore_index=True)
# Get total food waste per country from total_waste from previous analysis
total_waste_ghg = total_waste[['Country', 'Total Food Waste (tonnes/year)']].copy()

# Combine loss & waste, calculate GHG emissions
environmental_impact = pd.merge(total_loss_per_country, total_waste_ghg, on='Country', how='outer').fillna(0)
environmental_impact['Total Loss and Waste (tonnes)'] = (
    environmental_impact['Total Food Loss (tonnes/year)'] + environmental_impact['Total Food Waste (tonnes/year)']
)
environmental_impact['Estimated GHG Emissions (tonnes CO2eq)'] = (
    environmental_impact['Total Loss and Waste (tonnes)'] * GHG_EMISSION_FACTOR
)
environmental_impact = (
    environmental_impact
    .sort_values(by='Total Loss and Waste (tonnes)', ascending=False)
    .reset_index(drop=True)
)

# Format output
environmental_impact_fmt = environmental_impact.copy()
for col in [
    'Total Food Loss (tonnes/year)', 'Total Food Waste (tonnes/year)',
    'Total Loss and Waste (tonnes)', 'Estimated GHG Emissions (tonnes CO2eq)'
    ]:
    environmental_impact_fmt[col] = environmental_impact_fmt[col].map('{:,.0f}'.format)

print("--- Estimated Environmental Impact of Food Loss and Waste ---")
environmental_impact_fmt

--- Estimated Environmental Impact of Food Loss and Waste ---


,Country,Total Food Loss (tonnes/year),Total Food Waste (tonnes/year),Total Loss and Waste (tonnes),Estimated GHG Emissions (tonnes CO2eq)
0,Indonesia,"47,243,000","39,535,795","86,778,795","216,946,988"
1,Viet Nam,"9,520,000","15,384,854","24,904,854","62,262,135"
2,Thailand,"7,842,000","12,636,704","20,478,704","51,196,760"
3,Philippines,"4,756,000","12,728,801","17,484,801","43,712,002"
4,Myanmar,"5,076,000","8,804,564","13,880,564","34,701,410"
5,Malaysia,"2,062,000","7,110,919","9,172,919","22,932,298"
6,Cambodia,"1,621,000","2,838,261","4,459,261","11,148,152"
7,Lao People's Democratic Republic,"1,303,000","1,310,729","2,613,729","6,534,322"
8,Singapore,"342,000","893,813","1,235,813","3,089,532"
9,Brunei Darussalam,0,"70,577","70,577","176,442"


### Method: Using emissions per capita and population data (Waste Emissions)

In [134]:
asean_countries = [
    'Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao People\'s Democratic Republic',
    'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Viet Nam'
]

# Emissions per capita from waste (latest year per country)
emissions_per_capita = (
    fao_emissions[
        (fao_emissions['Area'].isin(asean_countries)) &
        (fao_emissions['Item'] == 'Waste') &
        (fao_emissions['Element'] == 'Emissions per capita') &
        (fao_emissions['Year'] == 2022)
    ]
    .rename(columns={'Area': 'Country', 'Year': 'Emission Data Year', 'Value': 'Emissions per Capita (tonnes CO2eq)'})
    [['Country', 'Emission Data Year', 'Emissions per Capita (tonnes CO2eq)']]
)

# Total population (2022)
total_population = (
    fao_population[
        (fao_population['Area'].isin(asean_countries)) &
        (fao_population['Element'] == 'Total Population - Both sexes') &
        (fao_population['Year'] == 2022)
    ]).rename(columns={'Area': 'Country', 'Year': 'Population Year', 'Value': 'Total Population'})
total_population['Total Population'] *= 1000
total_population = total_population[['Country', 'Population Year', 'Total Population']]

# Merge and calculate total emissions
ghg_via_per_capita = pd.merge(emissions_per_capita, total_population, on='Country')
ghg_via_per_capita['Total Emissions (tonnes CO2eq)'] = (
    ghg_via_per_capita['Emissions per Capita (tonnes CO2eq)'] * ghg_via_per_capita['Total Population']
)
ghg_via_per_capita = (
    ghg_via_per_capita
    .sort_values('Total Emissions (tonnes CO2eq)', ascending=False)
    .reset_index(drop=True)
)

ghg_via_per_capita_fmt = ghg_via_per_capita.copy()
ghg_via_per_capita_fmt['Total Emissions (tonnes CO2eq)'] = ghg_via_per_capita_fmt['Total Emissions (tonnes CO2eq)'].map('{:,.0f}'.format)
ghg_via_per_capita_fmt['Total Population'] = ghg_via_per_capita_fmt['Total Population'].map('{:,.0f}'.format)

print("--- Estimated GHG Emissions from Waste (Per Capita Method, tonnes) ---")
ghg_via_per_capita_fmt

--- Estimated GHG Emissions from Waste (Per Capita Method, tonnes) ---


,Country,Emission Data Year,Emissions per Capita (tonnes CO2eq),Population Year,Total Population,Total Emissions (tonnes CO2eq)
0,Indonesia,2022,0.33,2022,"278,830,529","92,014,075"
1,Thailand,2022,0.44,2022,"71,735,329","31,563,545"
2,Viet Nam,2022,0.28,2022,"99,680,655","27,910,583"
3,Philippines,2022,0.21,2022,"113,964,339","23,932,511"
4,Malaysia,2022,0.50,2022,"34,695,493","17,347,746"
5,Myanmar,2022,0.20,2022,"53,756,787","10,751,357"
6,Singapore,2022,0.85,2022,"5,649,885","4,802,402"
7,Cambodia,2022,0.20,2022,"17,201,724","3,440,345"
8,Lao People's Democratic Republic,2022,0.22,2022,"7,559,007","1,662,982"
9,Brunei Darussalam,2022,0.35,2022,"455,370","159,380"


### Method: Using emissions per capita and population data (Agrifood Systems Emissions)

In [135]:
asean_countries = [
    'Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao People\'s Democratic Republic',
    'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Viet Nam'
]

# Emissions per capita from waste (latest year per country)
emissions_per_capita = (
    fao_emissions[
        (fao_emissions['Area'].isin(asean_countries)) &
        (fao_emissions['Item'] == 'Agrifood systems') &
        (fao_emissions['Element'] == 'Emissions per capita') &
        (fao_emissions['Year'] == 2022)
    ].rename(columns={'Area': 'Country', 'Year': 'Emission Data Year', 'Value': 'Emissions per Capita (tonnes CO2eq)'})
    [['Country', 'Emission Data Year', 'Emissions per Capita (tonnes CO2eq)']]
)

# Merge and calculate total emissions
agrifood_system_ghg = pd.merge(emissions_per_capita, total_population, on='Country')
agrifood_system_ghg['Total Emissions (tonnes CO2eq)'] = (
    agrifood_system_ghg['Emissions per Capita (tonnes CO2eq)'] * agrifood_system_ghg['Total Population']
)
agrifood_system_ghg = (
    agrifood_system_ghg
    .sort_values('Total Emissions (tonnes CO2eq)', ascending=False)
    .reset_index(drop=True)
)

agrifood_system_ghg_fmt = agrifood_system_ghg.copy()
agrifood_system_ghg_fmt['Total Emissions (tonnes CO2eq)'] = agrifood_system_ghg_fmt['Total Emissions (tonnes CO2eq)'].map('{:,.0f}'.format)
agrifood_system_ghg_fmt['Total Population'] = agrifood_system_ghg_fmt['Total Population'].map('{:,.0f}'.format)

print("--- Estimated GHG Emissions from Waste (Per Capita Method, tonnes) ---")
agrifood_system_ghg_fmt

--- Estimated GHG Emissions from Waste (Per Capita Method, tonnes) ---


,Country,Emission Data Year,Emissions per Capita (tonnes CO2eq),Population Year,Total Population,Total Emissions (tonnes CO2eq)
0,Indonesia,2022,2.86,2022,"278,830,529","797,455,313"
1,Thailand,2022,2.59,2022,"71,735,329","185,794,502"
2,Myanmar,2022,3.29,2022,"53,756,787","176,859,829"
3,Viet Nam,2022,1.52,2022,"99,680,655","151,514,596"
4,Malaysia,2022,3.55,2022,"34,695,493","123,169,000"
5,Philippines,2022,0.93,2022,"113,964,339","105,986,835"
6,Cambodia,2022,3.63,2022,"17,201,724","62,442,258"
7,Lao People's Democratic Republic,2022,3.74,2022,"7,559,007","28,270,686"
8,Singapore,2022,0.82,2022,"5,649,885","4,632,906"
9,Brunei Darussalam,2022,3.66,2022,"455,370","1,666,654"


## Estimated Annual Electricity Generation Potential

In [136]:
# Assumptions and conversion factors based on the research report
# Isolate the inedible portion (e.g., peels, bones) of total food waste
# Based on studies of food composition, this is a defensible estimate
INEDIBLE_WASTE_PERCENTAGE = 0.60
# Step 1b: Account for waste that is not formally collected by municipal services
# This is a major constraint in the ASEAN region
WASTE_COLLECTION_EFFICIENCY = 0.47
# Biogas Yield - Based on the specific composition of food waste
BIOGAS_YIELD_M3_PER_TONNE = 184
# Methane Content - A realistic average for biogas from food waste
METHANE_PERCENTAGE_IN_BIOGAS = 0.60
# Energy Value - The standard Lower Heating Value (LHV) of pure methane
KWH_PER_M3_METHANE = 9.97
# Generator Efficiency - Reflects real-world operational efficiency for a CHP unit
GENERATOR_EFFICIENCY = 0.33

# Calculate total food loss and waste per country
total_loss_per_country.rename(columns={'Country': 'Country', 'Total Food Loss (tonnes/year)': 'Total Food Loss (tonnes)'}, inplace=True)
total_waste_df = total_loss_per_country.merge(
    total_waste[['Country', 'Total Food Waste (tonnes/year)']].rename(
        columns={'Total Food Waste (tonnes/year)': 'Total Food Waste (tonnes)'}
    ),
    on='Country', how='outer'
).fillna(0)
total_waste_df['Total Loss and Waste (tonnes)'] = (
    total_waste_df['Total Food Loss (tonnes)'] + total_waste_df['Total Food Waste (tonnes)']
)

# Calculate bioenergy potential
bioenergy = environmental_impact[['Country', 'Total Loss and Waste (tonnes)']].copy()
bioenergy['Available Feedstock (tonnes)'] = bioenergy['Total Loss and Waste (tonnes)'] * INEDIBLE_WASTE_PERCENTAGE * WASTE_COLLECTION_EFFICIENCY
bioenergy['Biogas Potential (m3)'] = bioenergy['Available Feedstock (tonnes)'] * BIOGAS_YIELD_M3_PER_TONNE
bioenergy['Methane Potential (m3)'] = bioenergy['Biogas Potential (m3)'] * METHANE_PERCENTAGE_IN_BIOGAS
bioenergy['Gross Energy Potential (kWh)'] = bioenergy['Methane Potential (m3)'] * KWH_PER_M3_METHANE
bioenergy['Net Electricity Generated (kWh)'] = bioenergy['Gross Energy Potential (kWh)'] * GENERATOR_EFFICIENCY

bioenergy = bioenergy.sort_values('Net Electricity Generated (kWh)', ascending=False).reset_index(drop=True)

# Format output
bioenergy_fmt = bioenergy.copy()
bioenergy_fmt['Total Loss and Waste (tonnes)'] = bioenergy['Total Loss and Waste (tonnes)'].map('{:,.0f}'.format)
bioenergy_fmt['Net Electricity Generated (kWh)'] = bioenergy['Net Electricity Generated (kWh)'].map('{:,.0f}'.format)
bioenergy_fmt['Available Feedstock (tonnes)'] = bioenergy['Available Feedstock (tonnes)'].map('{:,.0f}'.format)
bioenergy_fmt['Biogas Potential (m3)'] = bioenergy['Biogas Potential (m3)'].map('{:,.0f}'.format)
bioenergy_fmt['Methane Potential (m3)'] = bioenergy['Methane Potential (m3)'].map('{:,.0f}'.format)
bioenergy_fmt['Gross Energy Potential (kWh)'] = bioenergy['Gross Energy Potential (kWh)'].map('{:,.0f}'.format)

print("--- Estimated Annual Electricity Generation Potential from Inedible Food Waste ---")
bioenergy_fmt

--- Estimated Annual Electricity Generation Potential from Inedible Food Waste ---


,Country,Total Loss and Waste (tonnes),Available Feedstock (tonnes),Biogas Potential (m3),Methane Potential (m3),Gross Energy Potential (kWh),Net Electricity Generated (kWh)
0,Indonesia,"86,778,795","24,471,620","4,502,778,115","2,701,666,869","26,935,618,684","8,888,754,166"
1,Viet Nam,"24,904,854","7,023,169","1,292,263,064","775,357,839","7,730,317,651","2,551,004,825"
2,Thailand,"20,478,704","5,774,995","1,062,598,993","637,559,396","6,356,467,177","2,097,634,168"
3,Philippines,"17,484,801","4,930,714","907,251,354","544,350,813","5,427,177,601","1,790,968,608"
4,Myanmar,"13,880,564","3,914,319","720,234,705","432,140,823","4,308,444,004","1,421,786,521"
5,Malaysia,"9,172,919","2,586,763","475,964,421","285,578,653","2,847,219,167","939,582,325"
6,Cambodia,"4,459,261","1,257,512","231,382,135","138,829,281","1,384,127,930","456,762,217"
7,Lao People's Democratic Republic,"2,613,729","737,072","135,621,170","81,372,702","811,285,841","267,724,328"
8,Singapore,"1,235,813","348,499","64,123,865","38,474,319","383,588,960","126,584,357"
9,Brunei Darussalam,"70,577","19,903","3,662,099","2,197,260","21,906,678","7,229,204"


## Regulations (Questionable)

In [137]:
asean_countries = [
    'Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao People\'s Democratic Republic',
    'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Viet Nam'
]

# Household Waste per Capita
waste_per_capita = f_waste[f_waste['Country'].isin(asean_countries)][['Country', 'Household estimate (kg/capita/year)']].rename(
    columns={'Household estimate (kg/capita/year)': 'Household Waste (kg/capita/yr)'}
)

# Food Insecurity Prevalence
fs_insecurity = fao_fs[
    (fao_fs['Area'].isin(asean_countries)) &
    (fao_fs['Item'] == 'Prevalence of moderate or severe food insecurity in the total population (percent) (3-year average)') &
    (fao_fs['Element'] == 'Value') &
    (fao_fs['Year'].astype(str) == '2021-2023') 
].copy()[['Area', 'Value']].rename(
    columns={'Area': 'Country', 'Value': 'Food Insecurity (%)'}
)

# GHG Emissions per Capita
emissions_pc = fao_emissions[
    (fao_emissions['Area'].isin(asean_countries)) &
    (fao_emissions['Item'] == 'Waste') &
    (fao_emissions['Element'] == 'Emissions per capita') &
    (fao_emissions['Year']  == 2022)
].copy()[['Area', 'Value']].rename(
    columns={'Area': 'Country', 'Value': 'Waste GHG Emissions (tonnes CO2eq/capita)'}
)

# Assemble the Country Performance Scorecard
scorecard_df = waste_per_capita.merge(fs_insecurity, on='Country', how='left').merge(emissions_pc, on='Country', how='left')
scorecard_df = scorecard_df.sort_values('Food Insecurity (%)').reset_index(drop=True)

print("--- ASEAN Country Performance Scorecard ---")
scorecard_df

--- ASEAN Country Performance Scorecard ---


,Country,Household Waste (kg/capita/yr),Food Insecurity (%),Waste GHG Emissions (tonnes CO2eq/capita)
0,Viet Nam,72,10.8,0.28
1,Malaysia,81,16.7,0.50
2,Myanmar,78,32,0.20
3,Lao People's Democratic Republic,89,36.3,0.22
4,Indonesia,53,4.9,0.33
5,Philippines,26,44.1,0.21
6,Cambodia,85,50.5,0.20
7,Thailand,86,7.2,0.44
8,Singapore,68,7.7,0.85
9,Brunei Darussalam,76,NaN,0.35


In [138]:
# Load GFSI Data for Profiling
asean_gfsi_countries = ['Cambodia', 'Indonesia', 'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand']
gfsi_asean = gfsi_data[gfsi_data['Country'].isin(asean_gfsi_countries)]
gfsi_asean.loc[:, 'Value'] = gfsi_asean['Value'].astype(float)

# Display key GFSI indicators for top-performing countries
top_performers = scorecard_df.head(4)['Country'].tolist()
key_gfsi_indicators = [
    '1.5.1) Presence of food safety net programmes',
    '2.3.1) Crop storage facilities',
    '2.6.2) Road infrastructure',
    '2.9.1) Food security strategy',
    '2.5) Food loss'
]

profiling_df = gfsi_asean[
    (gfsi_asean['Country'].isin(top_performers)) &
    (gfsi_asean['Indicator'].isin(key_gfsi_indicators))
]

# Pivot table for comparison
profiling_pivot = profiling_df.pivot_table(index='Indicator', columns='Country', values='Value')

print("\n--- GFSI Profile of Countries with Lower Food Insecurity ---")
profiling_pivot


--- GFSI Profile of Countries with Lower Food Insecurity ---


Country,Malaysia,Myanmar
Indicator,,
1.5.1) Presence of food safety net programmes,1.0,1.0
2.3.1) Crop storage facilities,1.0,0.0
2.5) Food loss,1.66,6.34
2.6.2) Road infrastructure,2.0,0.0
2.9.1) Food security strategy,1.0,1.0


## General Data for ASEAN Countries

### Average dietary energy requirement in ASEAN

In [139]:
asean_diet_req = fao_fs[
    (fao_fs['Area'].isin(asean_countries)) &
    (fao_fs['Item'] == 'Average dietary energy requirement (kcal/cap/day)') &
    (fao_fs['Element'] == 'Value') &
    (fao_fs['Year'] == '2022')
][['Area', 'Value']].rename(
    columns={'Area': 'Country', 'Value': 'Average Dietary Energy Requirement (kcal/cap/day)'}
)

asean_diet_req['Average Dietary Energy Requirement (kcal/cap/day)'] = asean_diet_req['Average Dietary Energy Requirement (kcal/cap/day)'].astype(int)
asean_diet_req

,Country,Average Dietary Energy Requirement (kcal/cap/day)
27565,Brunei Darussalam,2423
32807,Cambodia,2302
91380,Indonesia,2316
106642,Lao People's Democratic Republic,2361
117367,Malaysia,2397
130676,Myanmar,2329
152169,Philippines,2258
172778,Singapore,2482
188516,Thailand,2447
209351,Viet Nam,2305


### Food supply (kcal/capita/day)

In [140]:
asean_food_supply = fao_fbs[
    (fao_fbs['Area'].isin(asean_countries)) &
    (fao_fbs['Element'] == 'Food supply (kcal/capita/day)') &
    (fao_fbs['Year'] == 2022)
][['Area', 'Value']].rename(
    columns={'Area': 'Country', 'Value': 'Food supply (kcal/capita/day)'}
)

asean_food_supply = asean_food_supply.groupby('Country').agg({'Food supply (kcal/capita/day)': 'sum'}).reset_index()
asean_food_supply

,Country,Food supply (kcal/capita/day)
0,Cambodia,2719.75
1,Indonesia,2785.96
2,Lao People's Democratic Republic,3182.63
3,Malaysia,3041.06
4,Myanmar,2821.22
5,Philippines,2851.74
6,Thailand,2933.00
7,Viet Nam,3105.38


### Food Waste per Capita

In [141]:
waste_cap_col = ['Household estimate (kg/capita/year)',
                 'Retail estimate (kg/capita/year)',
                 'Food service estimate (kg/capita/year)']

waste_per_capita = f_waste[f_waste['Country'].isin(asean_countries)][['Country'] + waste_cap_col].copy()

waste_per_capita

,Country,Household estimate (kg/capita/year),Retail estimate (kg/capita/year),Food service estimate (kg/capita/year)
30,Brunei Darussalam,76,46,35
35,Cambodia,85,44,40
98,Indonesia,53,50,40
113,Lao People's Democratic Republic,89,44,40
124,Malaysia,81,79,50
141,Myanmar,78,44,40
162,Philippines,26,44,40
188,Singapore,68,46,35
205,Thailand,86,50,40
227,Viet Nam,72,44,40


In [142]:
waste_per_capita.to_csv(r'../data/processed/waste_per_capita_columns.csv', index=False)

# Exporting Analysis Results

In [143]:
print("food_insec_asean columns:", food_insec_asean.columns.tolist())
print("financial_loss_by_country columns:", financial_loss_by_country.columns.tolist())
print("social_impact_household columns:", social_impact_household.columns.tolist())
print("social_impact_total columns:", social_impact_total.columns.tolist())
print("ghg_via_per_capita columns:", ghg_via_per_capita.columns.tolist())
print("bioenergy columns:", bioenergy.columns.tolist())
print("asean_diet_req columns:", asean_diet_req.columns.tolist())
print("asean_food_supply columns:", asean_food_supply.columns.tolist())
print("waste_per_capita columns:", waste_per_capita.columns.tolist())

food_insec_asean columns: ['Country', 'Population', 'Food Insecure Population', 'Food Insecurity (%)', 'Total Food Waste (tonnes/year)']
financial_loss_by_country columns: ['Country', 'Financial Loss (USD)']
social_impact_household columns: ['Country', 'People Fed Annually (from waste)', 'Food Insecure Population', 'Insecure Population Coverage (%)']
social_impact_total columns: ['Country', 'People Fed Annually (from waste)', 'Food Insecure Population', 'Insecure Population Coverage (%)']
ghg_via_per_capita columns: ['Country', 'Emission Data Year', 'Emissions per Capita (tonnes CO2eq)', 'Population Year', 'Total Population', 'Total Emissions (tonnes CO2eq)']
bioenergy columns: ['Country', 'Total Loss and Waste (tonnes)', 'Available Feedstock (tonnes)', 'Biogas Potential (m3)', 'Methane Potential (m3)', 'Gross Energy Potential (kWh)', 'Net Electricity Generated (kWh)']
asean_diet_req columns: ['Country', 'Average Dietary Energy Requirement (kcal/cap/day)']
asean_food_supply columns: ['

In [144]:
# Merge all dataframes on 'Country'
dfs_to_merge = [
    food_insec_asean,
    total_waste,
    total_waste_df,
    financial_loss_by_country,
    social_impact_total,
    ghg_via_per_capita,
    bioenergy,
    asean_diet_req,
    asean_food_supply,
    waste_per_capita
]

# Perform successive merges on 'Country'
merge_asean = reduce(lambda left, right: pd.merge(left, right, on='Country', how='outer'), dfs_to_merge)

print("--- Merged DataFrame ---")
merge_asean

--- Merged DataFrame ---


,Country,Population,Food Insecure Population_x,Food Insecurity (%),Total Food Waste (tonnes/year)_x,Household estimate (tonnes/year),Retail estimate (tonnes/year),Food service estimate (tonnes/year),Total Food Waste (tonnes/year)_y,Total Food Loss (tonnes),...,Available Feedstock (tonnes),Biogas Potential (m3),Methane Potential (m3),Gross Energy Potential (kWh),Net Electricity Generated (kWh),Average Dietary Energy Requirement (kcal/cap/day),Food supply (kcal/capita/day),Household estimate (kg/capita/year),Retail estimate (kg/capita/year),Food service estimate (kg/capita/year)
0,Brunei Darussalam,455370.0,NaN,NaN,70577,34109,20692,15776,70577,0.0,...,19902.714,3662099.376,2197259.6256,21906678.467232,7229203.894187,2423,NaN,76,46,35
1,Cambodia,17201724.0,8500000.0,50.5,2838261,1419831,742033,676397,2838261,1621000.0,...,1257511.602,231382134.768,138829280.8608,1384127930.182176,456762216.960118,2302,2719.75,85,44,40
2,Indonesia,278830529.0,13600000.0,4.9,39535795,14728364,13755721,11051710,39535795,47243000.0,...,24471620.19,4502778114.959999,2701666868.975999,26935618683.690716,8888754165.617937,2316,2785.96,53,50,40
3,Lao People's Democratic Republic,7559007.0,2700000.0,36.3,1310729,673831,333185,303713,1310729,1303000.0,...,737071.578,135621170.352,81372702.2112,811285841.045664,267724327.545069,2361,3182.63,89,44,40
4,Malaysia,34695493.0,5700000.0,16.7,7110919,2754808,2675062,1681049,7110919,2062000.0,...,2586763.158,475964421.072,285578652.6432,2847219166.852704,939582325.061392,2397,3041.06,81,79,50
5,Myanmar,53756787.0,17400000.0,32.0,8804564,4221946,2397336,2185282,8804564,5076000.0,...,3914319.048,720234704.832,432140822.8992,4308444004.305023,1421786521.420658,2329,2821.22,78,44,40
6,Philippines,113964339.0,51000000.0,44.1,12728801,2954580,5113255,4660966,12728801,4756000.0,...,4930713.882,907251354.288,544350812.5728,5427177601.350816,1790968608.445769,2258,2851.74,26,44,40
7,Singapore,5649885.0,500000.0,7.7,893813,409182,274980,209651,893813,342000.0,...,348499.266,64123864.944,38474318.9664,383588960.095008,126584356.831353,2482,NaN,68,46,35
8,Thailand,71735329.0,5200000.0,7.2,12636704,6180468,3579983,2876253,12636704,7842000.0,...,5774994.528,1062598993.152,637559395.8912,6356467177.035264,2097634168.421637,2447,2933.00,86,50,40
9,Viet Nam,99680655.0,10600000.0,10.8,15384854,7079811,4344674,3960369,15384854,9520000.0,...,7023168.828,1292263064.352,775357838.6112,7730317650.953664,2551004824.814709,2305,3105.38,72,44,40


In [145]:
merge_asean['Country'] = merge_asean['Country'].replace("Lao People's Democratic Republic", 'Laos')

merge_asean = merge_asean.replace(0, np.nan)

In [146]:
# Print all country names and the columns where the value is missing (NaN)
print("--- Missing Data by Country ---")
for idx, row in merge_asean.iterrows():
    missing_cols = row.index[row.isna()].tolist()
    if missing_cols:
        print(f"{row['Country']}: {missing_cols}")

--- Missing Data by Country ---
Brunei Darussalam: ['Food Insecure Population_x', 'Food Insecurity (%)', 'Total Food Loss (tonnes)', 'Financial Loss (USD)', 'Food Insecure Population_y', 'Insecure Population Coverage (%)', 'Food supply (kcal/capita/day)']
Singapore: ['Financial Loss (USD)', 'Food supply (kcal/capita/day)']


In [147]:
# Find columns in merge_asean that have exactly the same values for all rows
equal_cols = []
cols = merge_asean.columns.tolist()
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        col1, col2 = cols[i], cols[j]
        # Use np.nan for numeric columns, string for object columns
        fill1 = np.nan if pd.api.types.is_numeric_dtype(merge_asean[col1]) else '___NA___'
        fill2 = np.nan if pd.api.types.is_numeric_dtype(merge_asean[col2]) else '___NA___'
        s1 = merge_asean[col1].fillna(fill1).values
        s2 = merge_asean[col2].fillna(fill2).values
        # Compare values, treating NaNs as equal
        eq = ((s1 == s2) | (pd.isna(s1) & pd.isna(s2)))
        if merge_asean.shape[0] > 0 and np.all(eq):
            equal_cols.append((col1, col2))

print("Columns with identical values in merge_asean:")
for pair in equal_cols:
    print(pair)

Columns with identical values in merge_asean:
('Population', 'Total Population')
('Food Insecure Population_x', 'Food Insecure Population_y')
('Total Food Waste (tonnes/year)_x', 'Total Food Waste (tonnes/year)_y')
('Total Food Waste (tonnes/year)_x', 'Total Food Waste (tonnes)')
('Total Food Waste (tonnes/year)_y', 'Total Food Waste (tonnes)')
('Total Loss and Waste (tonnes)_x', 'Total Loss and Waste (tonnes)_y')
('Emission Data Year', 'Population Year')


In [148]:
# Remove redundant or duplicate columns from merge_asean
cols_to_drop = [
    'Emission Data Year', 'Population Year', 'Total Population',
    'Food Insecure Population_y',
    'Total Food Waste (tonnes/year)_x', 'Total Food Waste (tonnes/year)_y',
    'Total Loss and Waste (tonnes)_y'
]
merge_asean_no_dup = merge_asean.drop(cols_to_drop, axis=1)

In [149]:
merge_asean_no_dup.columns

Index(['Country', 'Population', 'Food Insecure Population_x',
       'Food Insecurity (%)', 'Household estimate (tonnes/year)',
       'Retail estimate (tonnes/year)', 'Food service estimate (tonnes/year)',
       'Total Food Loss (tonnes)', 'Total Food Waste (tonnes)',
       'Total Loss and Waste (tonnes)_x', 'Financial Loss (USD)',
       'People Fed Annually (from waste)', 'Insecure Population Coverage (%)',
       'Emissions per Capita (tonnes CO2eq)', 'Total Emissions (tonnes CO2eq)',
       'Available Feedstock (tonnes)', 'Biogas Potential (m3)',
       'Methane Potential (m3)', 'Gross Energy Potential (kWh)',
       'Net Electricity Generated (kWh)',
       'Average Dietary Energy Requirement (kcal/cap/day)',
       'Food supply (kcal/capita/day)', 'Household estimate (kg/capita/year)',
       'Retail estimate (kg/capita/year)',
       'Food service estimate (kg/capita/year)'],
      dtype='object')

In [150]:
# Rename columns for clarity and consistency
merge_asean_no_dup.rename(columns={
    'Food Insecure Population_x': 'Food Insecure Population',
    'Total Loss and Waste (tonnes)_x': 'Annual Food Loss and Waste (tonnes)',
    'Total Food Loss (tonnes)': 'Annual Food Loss (tonnes)',
    'Total Food Waste (tonnes)': 'Annual Food Waste (tonnes)',
    'Financial Loss (USD)': 'Annual Financial Loss from Food Loss (USD)',
    'People Fed Annually (from waste)': 'People Fed Annually from Edible Waste',
    'Total Emissions (tonnes CO2eq)': 'Annual GHG Emissions (tonnes CO2eq)',
    'Net Electricity Generated (kWh)': 'Annual Net Electricity from Inedible Waste (kWh)',
}, inplace=True)

merge_asean_relevant = merge_asean_no_dup.drop(['Emissions per Capita (tonnes CO2eq)',
                                                'Available Feedstock (tonnes)',
                                                'Biogas Potential (m3)',
                                                'Methane Potential (m3)',
                                                'Gross Energy Potential (kWh)'], axis=1)

In [151]:
# Manually filling in missing values for specific countries

# Singapore's financial loss from food loss based on the Singapore Environment Council Report (2019)
merge_asean_relevant.loc[merge_asean_relevant['Country'] == 'Singapore', 'Annual Financial Loss from Food Loss (USD)'] = 1833880000

# https://www.ceicdata.com/en/brunei/social-health-statistics/bn-prevalence-of-undernourishment--of-population (2020)
# merge_asean_relevant.loc[merge_asean_relevant['Country'] == 'Brunei Darussalam', 'Prevalence of Undernourishment (%)'] = 5.9
# brunei_insecure_pop = 5.9 / 100 * merge_asean_relevant[merge_asean_relevant['Country'] == 'Brunei Darussalam']['Population'].values[0]
# merge_asean_relevant.loc[merge_asean_relevant['Country'] == 'Brunei Darussalam', 'Food Insecure Population'] = brunei_insecure_pop

# https://openknowledge.fao.org/server/api/core/bitstreams/98ebf545-be51-45ff-a79d-fa1f2d55d71e/content (2011)
merge_asean_relevant.loc[merge_asean_relevant['Country'] == 'Brunei Darussalam', 'Food supply (kcal/capita/day)'] = 2949

# Mean Imputation
merge_asean_relevant.loc[merge_asean_relevant['Country'] == 'Singapore', 'Food supply (kcal/capita/day)'] = merge_asean_relevant['Food supply (kcal/capita/day)'].mean()

In [152]:
# Define ASEAN country coordinates (approximate centroids)
asean_coords = {
    'Brunei Darussalam': {'latitude': 4.5353, 'longitude': 114.7277},
    'Cambodia': {'latitude': 12.5657, 'longitude': 104.9910},
    'Indonesia': {'latitude': -0.7893, 'longitude': 113.9213},
    'Laos': {'latitude': 19.8563, 'longitude': 102.4955},
    "Lao People's Democratic Republic": {'latitude': 19.8563, 'longitude': 102.4955},
    'Malaysia': {'latitude': 4.2105, 'longitude': 101.9758},
    'Myanmar': {'latitude': 21.9162, 'longitude': 95.9560},
    'Philippines': {'latitude': 12.8797, 'longitude': 121.7740},
    'Singapore': {'latitude': 1.3521, 'longitude': 103.8198},
    'Thailand': {'latitude': 15.8700, 'longitude': 100.9925},
    'Viet Nam': {'latitude': 14.0583, 'longitude': 108.2772},
    'Vietnam': {'latitude': 14.0583, 'longitude': 108.2772}
}

# Map coordinates to the DataFrame
merge_asean_relevant['Latitude'] = merge_asean_relevant['Country'].map(lambda x: asean_coords.get(x, {}).get('latitude'))
merge_asean_relevant['Longitude'] = merge_asean_relevant['Country'].map(lambda x: asean_coords.get(x, {}).get('longitude'))

In [153]:
merge_asean_relevant.rename(columns={
    'Household estimate (tonnes/year)': 'Household FW estimate (tonnes/year)',
    'Retail estimate (tonnes/year)': 'Retail FW estimate (tonnes/year)',
    'Food service estimate (tonnes/year)': 'Food service FW estimate (tonnes/year)'
}, inplace=True)

In [154]:
merge_asean_relevant.columns

Index(['Country', 'Population', 'Food Insecure Population',
       'Food Insecurity (%)', 'Household FW estimate (tonnes/year)',
       'Retail FW estimate (tonnes/year)',
       'Food service FW estimate (tonnes/year)', 'Annual Food Loss (tonnes)',
       'Annual Food Waste (tonnes)', 'Annual Food Loss and Waste (tonnes)',
       'Annual Financial Loss from Food Loss (USD)',
       'People Fed Annually from Edible Waste',
       'Insecure Population Coverage (%)',
       'Annual GHG Emissions (tonnes CO2eq)',
       'Annual Net Electricity from Inedible Waste (kWh)',
       'Average Dietary Energy Requirement (kcal/cap/day)',
       'Food supply (kcal/capita/day)', 'Household estimate (kg/capita/year)',
       'Retail estimate (kg/capita/year)',
       'Food service estimate (kg/capita/year)', 'Latitude', 'Longitude'],
      dtype='object')

In [155]:
# Rearrange columns for better context and usability
ordered_cols = [
    'Country', 'Longitude', 'Latitude', 'Population',
    'Food Insecure Population', 'Food Insecurity (%)',
    'Annual Food Loss (tonnes)', 'Annual Food Waste (tonnes)', 'Annual Food Loss and Waste (tonnes)',
    'Household estimate (kg/capita/year)', 'Retail estimate (kg/capita/year)', 'Food service estimate (kg/capita/year)',
    'Household FW estimate (tonnes/year)', 'Retail FW estimate (tonnes/year)', 'Food service FW estimate (tonnes/year)',
    'Annual Financial Loss from Food Loss (USD)',
    'People Fed Annually from Edible Waste', 'Insecure Population Coverage (%)',
    'Annual GHG Emissions (tonnes CO2eq)',
    'Annual Net Electricity from Inedible Waste (kWh)',
    'Average Dietary Energy Requirement (kcal/cap/day)', 'Food supply (kcal/capita/day)'
]
merge_asean_relevant = merge_asean_relevant[ordered_cols]

In [156]:
# Ensure columns are unique before aggregation
merge_asean_relevant = merge_asean_relevant.loc[:, ~merge_asean_relevant.columns.duplicated()]

# Now add aggregation rows (sum and mean), ignoring Longitude and Latitude
agg_mean = merge_asean_relevant.drop(['Longitude', 'Latitude'], axis=1).select_dtypes(include=[np.number]).mean(numeric_only=True)
agg_sum = merge_asean_relevant.drop(['Longitude', 'Latitude'], axis=1).select_dtypes(include=[np.number]).sum(numeric_only=True)

mean_row = {col: agg_mean[col] if col in agg_mean else (np.nan if col in ['Longitude', 'Latitude'] else '') for col in merge_asean_relevant.columns}
sum_row = {col: agg_sum[col] if col in agg_sum else (np.nan if col in ['Longitude', 'Latitude'] else '') for col in merge_asean_relevant.columns}
mean_row['Country'] = 'ASEAN Mean'
sum_row['Country'] = 'ASEAN Sum'

merge_asean_relevant_with_aggs = pd.concat(
    [merge_asean_relevant, pd.DataFrame([mean_row, sum_row])],
    ignore_index=True
)

In [157]:
merge_asean_relevant_with_aggs

,Country,Longitude,Latitude,Population,Food Insecure Population,Food Insecurity (%),Annual Food Loss (tonnes),Annual Food Waste (tonnes),Annual Food Loss and Waste (tonnes),Household estimate (kg/capita/year),...,Household FW estimate (tonnes/year),Retail FW estimate (tonnes/year),Food service FW estimate (tonnes/year),Annual Financial Loss from Food Loss (USD),People Fed Annually from Edible Waste,Insecure Population Coverage (%),Annual GHG Emissions (tonnes CO2eq),Annual Net Electricity from Inedible Waste (kWh),Average Dietary Energy Requirement (kcal/cap/day),Food supply (kcal/capita/day)
0,Brunei Darussalam,114.7277,4.5353,455370.0,NaN,NaN,NaN,70577.0,70577.0,76.0,...,34109.0,20692.0,15776.0,NaN,64702.0,<NA>,1.593795e+05,7229203.894187,2423.0,2949.000000
1,Cambodia,104.9910,12.5657,17201724.0,8500000.0,50.500000,1.621000e+06,2838261.0,4459261.0,85.0,...,1419831.0,742033.0,676397.0,5.833735e+08,2730338.0,32.121624,3.440345e+06,456762216.960118,2302.0,2719.750000
2,Indonesia,113.9213,-0.7893,278830529.0,13600000.0,4.900000,4.724300e+07,39535795.0,86778795.0,53.0,...,14728364.0,13755721.0,11051710.0,1.567231e+10,37820470.0,278.091691,9.201407e+07,8888754165.617937,2316.0,2785.960000
3,Laos,102.4955,19.8563,7559007.0,2700000.0,36.300000,1.303000e+06,1310729.0,2613729.0,89.0,...,673831.0,333185.0,303713.0,6.137891e+08,1232562.0,45.650444,1.662982e+06,267724327.545069,2361.0,3182.630000
4,Malaysia,101.9758,4.2105,34695493.0,5700000.0,16.700000,2.062000e+06,7110919.0,9172919.0,81.0,...,2754808.0,2675062.0,1681049.0,9.460213e+08,6596490.0,115.727895,1.734775e+07,939582325.061392,2397.0,3041.060000
5,Myanmar,95.9560,21.9162,53756787.0,17400000.0,32.000000,5.076000e+06,8804564.0,13880564.0,78.0,...,4221946.0,2397336.0,2185282.0,3.619594e+09,8385172.0,48.190644,1.075136e+07,1421786521.420658,2329.0,2821.220000
6,Philippines,121.7740,12.8797,113964339.0,51000000.0,44.100000,4.756000e+06,12728801.0,17484801.0,26.0,...,2954580.0,5113255.0,4660966.0,1.905998e+09,12482694.0,24.475871,2.393251e+07,1790968608.445769,2258.0,2851.740000
7,Singapore,103.8198,1.3521,5649885.0,500000.0,7.700000,3.420000e+05,893813.0,1235813.0,68.0,...,409182.0,274980.0,209651.0,1.833880e+09,802265.0,160.453,4.802402e+06,126584356.831353,2482.0,2932.193333
8,Thailand,100.9925,15.8700,71735329.0,5200000.0,7.200000,7.842000e+06,12636704.0,20478704.0,86.0,...,6180468.0,3579983.0,2876253.0,2.358256e+09,11480498.0,220.778808,3.156354e+07,2097634168.421637,2447.0,2933.000000
9,Viet Nam,108.2772,14.0583,99680655.0,10600000.0,10.800000,9.520000e+06,15384854.0,24904854.0,72.0,...,7079811.0,4344674.0,3960369.0,4.562415e+09,14766744.0,139.308906,2.791058e+07,2551004824.814709,2305.0,3105.380000


In [158]:
merge_asean_relevant_with_aggs.to_excel('../data/final/asean_food_systems_analysis.xlsx', index=False)